In [ ]:
# 1. Proje Kurulumu ve Güncelleme
import os
import sys

# Google Colab kontrolü
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Google Colab ortamı algılandı.")
    # Drive mount (İsteğe bağlı, şu an repo kullanıyoruz)
    # drive.mount('/content/drive')
    
    # Repo klonlama / güncelleme
    repo_url = "https://github.com/onndd/newproje.git"
    project_dir = "/content/newproje"
    
    if os.path.exists(project_dir):
        print("Proje klasörü mevcut. Güncelleniyor...")
        %cd {project_dir}
        !git pull
    else:
        print("Proje klonlanıyor...")
        !git clone {repo_url}
        %cd {project_dir}
        
    # Gerekli kütüphaneler
    print("Kütüphaneler yükleniyor...")
    %pip install catboost hmmlearn
    
    # Path ayarı
    sys.path.append(project_dir)
else:
    print("Yerel ortam algılandı.")
    project_dir = os.getcwd()
    sys.path.append(project_dir)

In [ ]:
# 2. Veri Yükleme ve Hazırlık
import sqlite3
import pandas as pd
import numpy as np
from jetx_project.data_loader import load_data
from jetx_project.config import DB_PATH

print("Veri yükleniyor...")
try:
    df = load_data(DB_PATH)
    print(f"Toplam {len(df)} veri yüklendi.")
    print(df.head())
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen jetx.db dosyasının proje klasöründe olduğundan emin olun.")

In [ ]:
# 3. Psikolojik Modellerin Eğitimi (HMM)
print("Psikolojik Modeller (HMM) eğitiliyor...")
from jetx_project.model_hmm import train_hmm_model, predict_hmm_state, save_hmm_model

if 'df' in locals() and len(df) > 0:
    # HMM Eğitimi (Gözetimsiz)
    hmm_model, hmm_map = train_hmm_model(df['value'].values)
    save_hmm_model(hmm_model, hmm_map, output_dir='models')
    
    # Tüm geçmiş için durum tahmini (Feature olarak kullanmak için)
    hmm_states = hmm_model.predict(df['value'].values.reshape(-1, 1))
    # Durumları haritala (0=Soğuk, 1=Normal, 2=Sıcak)
    hmm_states_mapped = np.array([hmm_map[s] for s in hmm_states])
    
    print("HMM Durumları belirlendi.")
    print(f"Durum Dağılımı: {np.bincount(hmm_states_mapped)}")
else:
    print("Veri yok, HMM eğitilemedi.")

In [ ]:
# 4. Uzman Modellerin Eğitimi (Model A & B)
from jetx_project.model_a import prepare_model_a_data, train_model_a, save_models
from jetx_project.model_b import build_memory, train_model_b, save_memory

if 'df' in locals() and len(df) > 500:
    print("Model A için veri hazırlanıyor (Feature Extraction)...")
    # HMM durumlarını da feature olarak veriyoruz
    X, y_p15, y_p3, y_x = prepare_model_a_data(df['value'].values, hmm_states_mapped)
    
    print(f"Feature seti boyutu: {X.shape}")
    
    print("Model A eğitiliyor (CatBoost)...")
    model_p15, model_p3, model_x = train_model_a(X, y_p15, y_p3, y_x)
    save_models(model_p15, model_p3, model_x, output_dir='models')
    print("Model A eğitimi tamamlandı.")
    
    print("Model B (Hafıza) oluşturuluyor (k-NN)...")
    # Model B kendi içinde RTP ve Shockwave hesaplıyor
    patterns, targets = build_memory(df['value'].values)
    nbrs = train_model_b(patterns)
    save_memory(nbrs, patterns, targets, output_dir='models')
    print("Model B oluşturuldu.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 5. Simülasyon ve Test
from jetx_project.simulation import run_simulation

if 'df' in locals() and len(df) > 1000:
    print("Simülasyon başlıyor...")
    
    # Test verisi (Son 2000 el)
    test_df = df.iloc[-2000:].copy()
    
    # Model A Simülasyonu
    print("Model A tahminleri üretiliyor...")
    # Not: Simülasyon fonksiyonunu HMM desteği ile güncellememiz gerekecek.
    # Şimdilik mevcut haliyle çalıştırıyoruz, HMM feature'ları eğitimde kullanıldı.
    # Ancak canlı simülasyonda HMM state'ini anlık tahmin etmemiz lazım.
    # Bu basit testte, eğitimdeki gibi toplu feature çıkarımı yapıp test edeceğiz.
    
    # HMM state'lerini test verisi için al
    test_hmm_states = hmm_states_mapped[-2000:]
    
    # Model A için featureları tekrar çıkar (Test seti için)
    X_test, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=len(df)-2000)
    
    # Tahminler
    probs_p15 = model_p15.predict_proba(X_test)[:, 1]
    probs_p3 = model_p3.predict_proba(X_test)[:, 1]
    preds_x = model_x.predict(X_test)
    
    # Sonuçları DataFrame'e ekle
    # Dikkat: X_test boyutu ile test_df boyutu eşleşmeli (start_index nedeniyle 1 eksik olabilir)
    # prepare_model_a_data N-1 tane döndürür.
    
    sim_results_a = test_df.iloc[:-1].copy()
    sim_results_a['prob_1.5'] = probs_p15
    sim_results_a['prob_3.0'] = probs_p3
    sim_results_a['pred_value'] = preds_x
    
    print("--- Simulation Results for Model A ---")
    run_simulation(sim_results_a, model_name="Model A (Psych)")
    
    # Model B Simülasyonu
    print("Model B tahminleri üretiliyor...")
    # Model B zaten anlık çalışıyor, simülasyon fonksiyonu içinde çağrılıyor mu?
    # Hayır, run_simulation hazır sonuçları bekliyor. 
    # Model B için manuel döngü lazım veya evaluate fonksiyonu.
    # Şimdilik Model A'ya odaklanalım.
    
else:
    print("Simülasyon için yeterli veri yok.")

In [ ]:
# 6. Sonuçları İndir (Opsiyonel)
!zip -r models.zip models/
from google.colab import files
# files.download('models.zip')